# DP-DKI analysis

In [1]:
#---------------------------------------------------------------------
# Package Management
#---------------------------------------------------------------------
import sys as sys
import os # mkdir
import os.path as op # path
import glob as glob
from shutil import copyfile, rmtree
import subprocess
import pandas as pd
from mat73 import loadmat
import xml.etree.ElementTree as ET
import math

import numpy as np # array, ndarray
import numpy.matlib as npm

import nibabel as nib

import scipy as sp
from scipy import special
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline
from scipy.interpolate import interp2d
from scipy.ndimage import generate_binary_structure
from scipy.ndimage import binary_erosion

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import warnings
warnings.filterwarnings('ignore')

### Define things...

In [ ]:
# subj = 'IAM_DEV_002'
# root = '/home/hum200/data/IAM_renewal/' + subj
# dcmPath = op.join(root,'dicom')

# dpdkib1r1Dir = op.join(root,'DPDKI_b1_r1')
# dpdkib1r2Dir = op.join(root,'DPDKI_b1_r2')

# dpdkib2r1Dir = op.join(root,'DPDKI_b2_r1')
# dpdkib2r2Dir = op.join(root,'DPDKI_b2_r2')

# dpdki63Dir = op.join(root,'DPDKI63')

# dpdkiDir = op.join(root,'DPDKI')

# dpdki_all = op.join(dpdki63Dir,'dwi_preprocessed.nii')

# b1_run1 = op.join(dpdkib1r1Dir,'dwi_preprocessed.nii')
# b1_run2 = op.join(dpdkib1r2Dir,'dwi_preprocessed.nii')

# b2_run1 = op.join(dpdkib2r1Dir,'dwi_preprocessed.nii')
# b2_run2 = op.join(dpdkib2r2Dir,'dwi_preprocessed.nii')

# dpdki_b1 = op.join(dpdkiDir,'dpdki_b1.nii')
# dpdki_b2 = op.join(dpdkiDir,'dpdki_b2.nii')
# dpdki63 = op.join(dpdkiDir,'dpdki63.nii')

# preprocessFlg = 1
# fwhm = str(1.5)

In [2]:
subj = 'DDE_v4_QA'
root = '/home/hum200/data/IAM_renewal/' + subj
dcmPath = op.join(root,'dicom')

dpdkib1r1te115Dir = op.join(root,'DPDKI_b1_r1_te115')
dpdkib1r2te115Dir = op.join(root,'DPDKI_b1_r2_te115')
dpdkib1r3te115Dir = op.join(root,'DPDKI_b1_r3_te115')

dpdkib2r1te115Dir = op.join(root,'DPDKI_b2_r1_te115')
dpdkib2r2te115Dir = op.join(root,'DPDKI_b2_r2_te115')
dpdkib2r3te115Dir = op.join(root,'DPDKI_b2_r3_te115')

# Longer TE = 140ms for potentially better exchange rate estimate
dpdkib1r1te140Dir = op.join(root,'DPDKI_b1_r1_te140')
dpdkib1r2te140Dir = op.join(root,'DPDKI_b1_r2_te140')
dpdkib1r3te140Dir = op.join(root,'DPDKI_b1_r3_te140')
dpdkib1r4te140Dir = op.join(root,'DPDKI_b1_r4_te140')

dpdkib2r1te140Dir = op.join(root,'DPDKI_b2_r1_te140')
dpdkib2r2te140Dir = op.join(root,'DPDKI_b2_r2_te140')
dpdkib2r3te140Dir = op.join(root,'DPDKI_b2_r3_te140')
dpdkib2r4te140Dir = op.join(root,'DPDKI_b2_r4_te140')

dpdkiDirte115 = op.join(root,'DPDKI_TE115')
dpdkiDirte140 = op.join(root,'DPDKI_TE140')

b1_run1_te115 = op.join(dpdkib1r1te115Dir,'dwi_preprocessed.nii')
b1_run2_te115 = op.join(dpdkib1r2te115Dir,'dwi_preprocessed.nii')
b1_run3_te115 = op.join(dpdkib1r3te115Dir,'dwi_preprocessed.nii')

b2_run1_te115 = op.join(dpdkib2r1te115Dir,'dwi_preprocessed.nii')
b2_run2_te115 = op.join(dpdkib2r2te115Dir,'dwi_preprocessed.nii')
b2_run3_te115 = op.join(dpdkib2r3te115Dir,'dwi_preprocessed.nii')

# dpdki_b1_te115 = op.join(dpdkiDir,'dpdki_b1_te115.nii')
# dpdki_b2_te115 = op.join(dpdkiDir,'dpdki_b2_te115.nii')

b1_run1_te140 = op.join(dpdkib1r1te140Dir,'dwi_preprocessed.nii')
b1_run2_te140 = op.join(dpdkib1r2te140Dir,'dwi_preprocessed.nii')
b1_run3_te140 = op.join(dpdkib1r3te140Dir,'dwi_preprocessed.nii')
b1_run4_te140 = op.join(dpdkib1r4te140Dir,'dwi_preprocessed.nii')

b2_run1_te140 = op.join(dpdkib2r1te140Dir,'dwi_preprocessed.nii')
b2_run2_te140 = op.join(dpdkib2r2te140Dir,'dwi_preprocessed.nii')
b2_run3_te140 = op.join(dpdkib2r3te140Dir,'dwi_preprocessed.nii')
b2_run4_te140 = op.join(dpdkib2r4te140Dir,'dwi_preprocessed.nii')

# dpdki_b1_te140 = op.join(dpdkiDir,'dpdki_b1_te140.nii')
# dpdki_b2_te140 = op.join(dpdkiDir,'dpdki_b2_te140.nii')


preprocessFlg = 0
fwhm = str(1.5)

### Prepocessing for the DP-DKI data

In [ ]:
# if preprocessFlg == 1:
    
#     os.makedirs(dpdki63Dir, exist_ok=True)

#     dwiIn = op.join(dcmPath,'DPDKI_63dir_TE115_20')
#     dwiDir = dpdki63Dir

#     arg = [
#         'pydesigner','-n',
#         '-z','--fwhm',fwhm,
#         '--mask','-cf',
#         '-o',dwiDir,
#         dwiIn,
#         '--nofit',
#         '--force'
#         ]
#     completion = subprocess.run(arg)

#     os.makedirs(dpdkib1r1Dir, exist_ok=True)

#     dwiIn = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run1_21')
#     dwiDir = dpdkib1r1Dir

#     arg = [
#         'pydesigner','-n',
#         '-z','--fwhm',fwhm,
#         '--mask','-cf',
#         '-o',dwiDir,
#         dwiIn,
#         '--nofit',
#         '--force'
#         ]
#     completion = subprocess.run(arg)

#     os.makedirs(dpdkib1r2Dir, exist_ok=True)

#     dwiIn = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run2_23')
#     dwiDir = dpdkib1r2Dir

#     arg = [
#         'pydesigner','-n',
#         '-z','--fwhm',fwhm,
#         '--mask','-cf',
#         '-o',dwiDir,
#         dwiIn,
#         '--nofit',
#         '--force'
#         ]
#     completion = subprocess.run(arg)

#     os.makedirs(dpdkib2r1Dir, exist_ok=True)

#     dwiIn = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run1_22')
#     dwiDir = dpdkib2r1Dir

#     arg = [
#         'pydesigner','-n',
#         '-z','--fwhm',fwhm,
#         '--mask','-cf',
#         '-o',dwiDir,
#         dwiIn,
#         '--nofit',
#         '--force'
#         ]
#     completion = subprocess.run(arg)

#     os.makedirs(dpdkib2r2Dir, exist_ok=True)

#     dwiIn = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run2_24')
#     dwiDir = dpdkib2r2Dir

#     arg = [
#         'pydesigner','-n',
#         '-z','--fwhm',fwhm,
#         '--mask','-cf',
#         '-o',dwiDir,
#         dwiIn,
#         '--nofit',
#         '--force'
#         ]
#     completion = subprocess.run(arg)

# #     fwhm = 2
# #     stddev = fwhm / np.sqrt(8 * np.log(2))
# #     print(stddev)

#     # stddev = 2
#     # fwhm = stddev * np.sqrt(8 * np.log(2))
#     # print(fwhm)


#     os.makedirs(dpdkiDir, exist_ok=True)

#     arg = [
#         'mrcalc',
#         b1_run1,
#         b1_run2,
#         '-add',
#         '2','-div',
#         dpdki_b1
#     ]
#     completion = subprocess.run(arg)

#     arg = [
#         'mrcalc',
#         b2_run1,
#         b2_run2,
#         '-add',
#         '2','-div',
#         dpdki_b2
#     ]
#     completion = subprocess.run(arg)

#     copyfile(dpdki_all,dpdki63)


In [3]:
if preprocessFlg == 1:

    os.makedirs(dpdkiDirte115, exist_ok=True)
    
    dwiIn1 = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run1_13')
    dwiIn2 = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run2_15')
    dwiIn3 = op.join(dcmPath,'DPDKI_9dir_TE115_b1000_run3_17')
    
    dwiIn4 = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run1_14')
    dwiIn5 = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run2_16')
    dwiIn6 = op.join(dcmPath,'DPDKI_39dir_TE115_b2000_run3_18')
    
    dwiDir = dpdkiDirte115

    arg = [
        'pydesigner','-s',
        '--fwhm',fwhm,
        '--mask',
        '-o',dwiDir,
        dwiIn1,dwiIn2,dwiIn3,
        dwiIn4,dwiIn5,dwiIn6,
        '--nofit',
        '--force'
        ]
    completion = subprocess.run(arg)
    
    

In [4]:
if preprocessFlg == 1:
    
    os.makedirs(dpdkiDirte140, exist_ok=True)
    
    dwiIn1 = op.join(dcmPath,'DPDKI_9dir_TE140_b1000_run1_5')
    dwiIn2 = op.join(dcmPath,'DPDKI_9dir_TE140_b1000_run2_7')
    dwiIn3 = op.join(dcmPath,'DPDKI_9dir_TE140_b1000_run3_9')
    dwiIn4 = op.join(dcmPath,'DPDKI_9dir_TE140_b1000_run4_11')
    
    dwiIn5 = op.join(dcmPath,'DPDKI_39dir_TE140_b2000_run1_6')
    dwiIn6 = op.join(dcmPath,'DPDKI_39dir_TE140_b2000_run2_8')
    dwiIn7 = op.join(dcmPath,'DPDKI_39dir_TE140_b2000_run3_10')
    dwiIn8 = op.join(dcmPath,'DPDKI_39dir_TE140_b2000_run4_12')
    
    dwiDir = dpdkiDirte140

#     arg = [
#         'pydesigner','-n',
#         '-z','--fwhm',fwhm,
#         '--mask','-cf',
#         '-o',dwiDir,
#         dwiIn1,dwiIn2,dwiIn3,dwiIn4,
#         dwiIn5,dwiIn6,dwiIn7,dwiIn8,
#         '--nofit',
#         '--force'
#         ]

    arg = [
            'pydesigner','-s',
            '--fwhm',fwhm,
            '--mask',
            '-o',dwiDir,
            dwiIn1,dwiIn2,dwiIn3,dwiIn4,
            dwiIn5,dwiIn6,dwiIn7,dwiIn8,
            '--nofit',
            '--force'
    ]
    completion = subprocess.run(arg)
    
    

### Calculate the diffusivity plus 4 kurtosis metrics of DP-DKI

In [ ]:
# arg = [
#     'fslmaths',
#     op.join(dpdki63Dir,'brain_mask.nii'),
#     '-eroF',
#     op.join(dpdkiDir,'brain_mask_eroF'),
#     ]
# completion = subprocess.run(arg)

# brain_img = nib.load(op.join(dpdkiDir,'brain_mask_eroF.nii.gz'))
# # csf_img = nib.load(op.join(dpdki63Dir,'csf_mask.nii'))

# mask = brain_img.get_fdata()
# # mask[csf_img.get_fdata() == 1] = 0


# dpdkib1_img = nib.load(dpdki_b1)
# dpdkib2_img = nib.load(dpdki_b2)
# dpdki63_img = nib.load(dpdki63)

# # Load bvecs
# bvecs63 = np.loadtxt(op.join(dpdki63Dir,'dwi_preprocessed.bvec'))
# bvecsb1 = np.loadtxt(op.join(dpdkib1r1Dir,'dwi_preprocessed.bvec'))
# bvecsb2 = np.loadtxt(op.join(dpdkib2r1Dir,'dwi_preprocessed.bvec'))
# # Load bvals
# bvals63 = np.rint(np.loadtxt(op.join(dpdki63Dir,'dwi_preprocessed.bval')))
# bvalsb1 = np.rint(np.loadtxt(op.join(dpdkib1r1Dir,'dwi_preprocessed.bval')))
# bvalsb2 = np.rint(np.loadtxt(op.join(dpdkib2r1Dir,'dwi_preprocessed.bval')))

# S0b1 = np.mean(dpdkib1_img.get_fdata()[:,:,:,bvalsb1 == 0],axis = 3)
# Sb1 = dpdkib1_img.get_fdata()[:,:,:,bvalsb1 != 0]

# S0b2 = np.mean(dpdkib2_img.get_fdata()[:,:,:,bvalsb2 == 0],axis = 3)
# Sb2 = dpdkib2_img.get_fdata()[:,:,:,bvalsb2 != 0]

# S063 = np.mean(dpdki63_img.get_fdata()[:,:,:,bvals63 == 0],axis = 3)
# S63b1 = dpdki63_img.get_fdata()[:,:,:,bvals63 == 1000]
# S63b2 = dpdki63_img.get_fdata()[:,:,:,bvals63 == 2000]

# psi0 = np.log((S0b1 + S0b2 + S063)/3)
# psi0[np.isnan(psi0)] = 0
# dims = np.shape(psi0)
# psi0 = np.reshape(psi0,np.prod(dims), order = 'F')

# Sb1 = (Sb1 + S63b1[:,:,:,0:9])/2
# Sb1[np.isnan(Sb1)] = 0
# Sb1 = np.reshape(Sb1,(np.prod(dims),9), order = 'F')

# Sb2 = (Sb2 + S63b2[:,:,:,0:39])/2
# Sb2[np.isnan(Sb2)] = 0
# Sb2 = np.reshape(Sb2,(np.prod(dims),39), order = 'F') 

# AmbL = np.log(Sb1)
# AmbH = np.log(Sb2)

# S63b1[np.isnan(S63b1)] = 0
# S63b1 = np.reshape(S63b1,(np.prod(dims),63), order = 'F')

# S63b2[np.isnan(S63b2)] = 0
# S63b2 = np.reshape(S63b2,(np.prod(dims),63), order = 'F')

# Am63b1 = np.log(S63b1)
# Am63b2 = np.log(S63b2)

# AmbL[np.isnan(AmbL)] = 0
# AmbH[np.isnan(AmbH)] = 0

# Am63b1[np.isnan(Am63b2)] = 0
# Am63b2[np.isnan(Am63b2)] = 0

# # Eq. 19 for b = 1000 and b = 2000
# psibL = (1/15) * np.sum(AmbL[:,0:3], axis = 1) + (2/15) * np.sum(AmbL[:,3:9],axis = 1)
# psibH = (1/15) * np.sum(AmbH[:,0:3], axis = 1) + (2/15) * np.sum(AmbH[:,3:9],axis = 1)

# # Eq. 20 for b = 2000
# psibHt = (1/12) * np.sum(AmbH[:,3:15],axis = 1) + (1/24) * np.sum(AmbH[:,15:21],axis = 1) - (1/12) * np.sum(AmbH[:,0:3],axis = 1)
# psibHt[np.isnan(psibHt)] = 0
# psibHt[np.isinf(psibHt)] = 0

# dpsibHt = psibH - psibHt # Eq. 21 for b = 2000
# dpsibHt[np.isnan(dpsibHt)] = 0
# dpsibHt[np.isinf(dpsibHt)] = 0

# # Eq.22 for b = 2000
# dpsibHAt = (2/5)*np.sum(AmbH[:,0:3],axis = 1) + (2/15)*np.sum(AmbH[:,3:9], axis = 1) - (2/15)*np.sum(AmbH[:,9:15], axis = 1) - (1/5)*np.sum(AmbH[:,15:21], axis = 1) - (1/15)*np.sum(AmbH[:,21:33], axis = 1) + (2/15)*np.sum(AmbH[:,33:39], axis = 1)
# dpsibHAt[np.isnan(dpsibHAt)] = 0
# dpsibHAt[np.isinf(dpsibHAt)] = 0

# # Eq. 36 b = 2000
# m = np.arange(16,22,1,dtype = 'int')

# psibHCt = (1/6)*np.sum((-1)**m * AmbH[:,15:21],axis = 1)
# psibHCt[np.isnan(psibHCt)] = 0
# psibHCt[np.isinf(psibHCt)] = 0

# Dbar = (3/2)*psi0 - 2*psibL + (1/2)*psibH #Eq. 31
# Wbar = (3/Dbar**2)*(psi0 - 2*psibL + psibH) #Eq. 32

# dWbar = (6/(Dbar**2 * 2**2)) * dpsibHt # Eq. 33
# dWAt = (6/(Dbar**2 * 2**2)) * dpsibHAt # Eq. 34

# Wbart = Wbar - dWbar
# WAt = Wbar - dWAt

# # Cbar1 = -(1/2) * psibHCt # Eq. 35

# psibLt2 = (3/40)*np.sum(Am63b1[:,39:51],axis = 1) - (3/40)*np.sum(Am63b1[:,51:63],axis = 1) - (1/30)*np.sum((-1)**m * Am63b1[:,15:21],axis = 1)
# psibHt2 = (3/40)*np.sum(Am63b2[:,39:51],axis = 1) - (3/40)*np.sum(Am63b2[:,51:63],axis = 1) - (1/30)*np.sum((-1)**m * Am63b2[:,15:21],axis = 1)

# Cbar = ((1/2)*psibHt2 - psibLt2) # Eq. 37
# WBt = (12/(Dbar**2)) * ((1/2)*psibHt2 - psibLt2)

# Dbar = np.reshape(Dbar,(dims[0],dims[1],dims[2]),order = 'F')
# Wbar = np.reshape(Wbar,(dims[0],dims[1],dims[2]),order = 'F')
# Wbart = np.reshape(Wbart,(dims[0],dims[1],dims[2]),order = 'F')
# WAt = np.reshape(WAt,(dims[0],dims[1],dims[2]),order = 'F')
# Cbar = np.reshape(Cbar,(dims[0],dims[1],dims[2]),order = 'F')
# WBt = np.reshape(WBt,(dims[0],dims[1],dims[2]),order = 'F')

# Dbar[mask == 0] = 0
# Wbar[mask == 0] = 0
# Wbart[mask == 0] = 0
# WAt[mask == 0] = 0
# Cbar[mask == 0] = 0
# WBt[mask == 0] = 0

# Wbar[Wbar < 0] = 0
# Wbart[Wbart < 0] = 0
# WAt[WAt < 0] = 0

# outname = op.join(dpdkiDir,'Dbar.nii')
# newimg = nib.Nifti1Image(Dbar, dpdki63_img.affine, dpdki63_img.header)
# nib.save(newimg, outname)

# outname = op.join(dpdkiDir,'Wbar.nii')
# newimg = nib.Nifti1Image(Wbar, dpdki63_img.affine, dpdki63_img.header)
# nib.save(newimg, outname)

# outname = op.join(dpdkiDir,'Wbart.nii')
# newimg = nib.Nifti1Image(Wbart, dpdki63_img.affine, dpdki63_img.header)
# nib.save(newimg, outname)

# outname = op.join(dpdkiDir,'WAt.nii')
# newimg = nib.Nifti1Image(WAt, dpdki63_img.affine, dpdki63_img.header)
# nib.save(newimg, outname)

# outname = op.join(dpdkiDir,'Cbar.nii')
# newimg = nib.Nifti1Image(Cbar, dpdki63_img.affine, dpdki63_img.header)
# nib.save(newimg, outname)
       
# outname = op.join(dpdkiDir,'WBt.nii')
# newimg = nib.Nifti1Image(WBt, dpdki63_img.affine, dpdki63_img.header)
# nib.save(newimg, outname)

In [18]:
# dpdkiDir = dpdkiDirte115
dpdkiDir = dpdkiDirte140

hdr = nib.load(op.join(dpdkiDir,'dwi_preprocessed.nii'))
imgIn = hdr.get_fdata()
bvecIn = np.loadtxt(op.join(dpdkiDir,'dwi_preprocessed.bvec')).T
bvalIn = np.loadtxt(op.join(dpdkiDir,'dwi_preprocessed.bval')).T

idx0 = np.where(bvalIn == 0)[0]
idx9 = np.where(bvalIn == 1000)[0]
idx39 = np.where(bvalIn == 2000)[0]

dims = np.shape(imgIn)[0:3]

b0 = imgIn[:,:,:,idx0]
b1 = imgIn[:,:,:,idx9]
b2 = imgIn[:,:,:,idx39]

b0 = np.reshape(b0,(np.prod(dims),len(idx0)),order = 'F')
b1 = np.reshape(b1,(np.prod(dims),9,4),order = 'F')
b2 = np.reshape(b2,(np.prod(dims),39,4),order = 'F')

b0 = np.mean(b0, axis = 1)
b1 = np.mean(b1, axis = 2)
b2 = np.mean(b2, axis = 2)

arg = [
    'fslmaths',
    op.join(dpdkiDir,'brain_mask.nii'),
    '-eroF',
    op.join(dpdkiDir,'brain_mask_eroF'),
    ]
completion = subprocess.run(arg)

brain_img = nib.load(op.join(dpdkiDir,'brain_mask_eroF.nii.gz'))

mask = brain_img.get_fdata()

psi0 = np.log(b0)
psi0[np.isnan(psi0)] = 0 

Sb1 = b1
Sb2 = b2

AmbL = np.log(Sb1)
AmbH = np.log(Sb2)

AmbL[np.isnan(AmbL)] = 0
AmbH[np.isnan(AmbH)] = 0

# Eq. 19 for b = 1000 and b = 2000
psibL = (1/15) * np.sum(AmbL[:,0:3], axis = 1) + (2/15) * np.sum(AmbL[:,3:9],axis = 1)
psibH = (1/15) * np.sum(AmbH[:,0:3], axis = 1) + (2/15) * np.sum(AmbH[:,3:9],axis = 1)

# Eq. 20 for b = 2000
psibHt = (1/12) * np.sum(AmbH[:,3:15],axis = 1) + (1/24) * np.sum(AmbH[:,15:21],axis = 1) - (1/12) * np.sum(AmbH[:,0:3],axis = 1)
psibHt[np.isnan(psibHt)] = 0
psibHt[np.isinf(psibHt)] = 0

dpsibHt = psibH - psibHt # Eq. 21 for b = 2000
dpsibHt[np.isnan(dpsibHt)] = 0
dpsibHt[np.isinf(dpsibHt)] = 0

# Eq.22 for b = 2000
dpsibHAt = (2/5)*np.sum(AmbH[:,0:3],axis = 1) + (2/15)*np.sum(AmbH[:,3:9], axis = 1) - (2/15)*np.sum(AmbH[:,9:15], axis = 1) - (1/5)*np.sum(AmbH[:,15:21], axis = 1) - (1/15)*np.sum(AmbH[:,21:33], axis = 1) + (2/15)*np.sum(AmbH[:,33:39], axis = 1)
dpsibHAt[np.isnan(dpsibHAt)] = 0
dpsibHAt[np.isinf(dpsibHAt)] = 0

# Eq. 36 b = 2000
m = np.arange(16,22,1,dtype = 'int')

psibHCt = (1/6)*np.sum((-1)**m * AmbH[:,15:21],axis = 1)
psibHCt[np.isnan(psibHCt)] = 0
psibHCt[np.isinf(psibHCt)] = 0

Dbar = (3/2)*np.squeeze(psi0) - 2*psibL + (1/2)*psibH #Eq. 31
Wbar = (3/Dbar**2)*(np.squeeze(psi0) - 2*psibL + psibH) #Eq. 32

dWbar = (6/(Dbar**2 * 2**2)) * dpsibHt # Eq. 33
dWAt = (6/(Dbar**2 * 2**2)) * dpsibHAt # Eq. 34

Wbart = Wbar - dWbar
WAt = Wbar - dWAt


# Calculate the associated anisotropies
# First the FA

G = (1/2) * ((AmbL[:,3] - AmbL[:,4])**2 + (AmbL[:,5] - AmbL[:,6])**2 + (AmbL[:,7] - AmbL[:,8])**2) # Eq. 42
numer = ((AmbL[:,0] - AmbL[:,1])**2 + (AmbL[:,0] - AmbL[:,2])**2 + (AmbL[:,1] - AmbL[:,2])**2 + 3*G)
denom = ((psi0 - AmbL[:,0])**2 + (psi0 - AmbL[:,1])**2 + (psi0 - AmbL[:,2])**2 + G)

fa = np.sqrt(numer/(2*denom)) # Eq. 41

# Type-II microFA
muFAprime = np.sqrt((30*dWbar)/(9 + 20*dWbar)) #Eq. 39

# Type-I microFA
muFA = np.sqrt(3 * (3*fa**2 + 3*muFAprime**2 - 4 * fa**2 * muFAprime**2)/(9 - 4 * fa**2 * muFAprime**2)) #Eq. 40


# Calculate the exchange rate
Delta = 24.7 * 10**-3
# tau = 28.1 * 10**-3

tau = (28.1 + 27) * 10**-3
Re = (3/(2*Delta + 3*tau))*np.log(Wbar/WAt)


# Reshape and read-out the images 

Dbar = np.reshape(Dbar,(dims[0],dims[1],dims[2]),order = 'F')
Wbar = np.reshape(Wbar,(dims[0],dims[1],dims[2]),order = 'F')
Wbart = np.reshape(Wbart,(dims[0],dims[1],dims[2]),order = 'F')
WAt = np.reshape(WAt,(dims[0],dims[1],dims[2]),order = 'F')

Dbar[mask == 0] = 0
Wbar[mask == 0] = 0
Wbart[mask == 0] = 0
WAt[mask == 0] = 0

# Wbar[Wbar < 0] = 0
# Wbar[Wbar > 3] = 3
# Wbart[Wbart < 0] = 0
# WAt[WAt < 0] = 0

fa = np.reshape(fa,(dims[0],dims[1],dims[2]),order = 'F')
fa[mask == 0] = 0

muFAprime = np.reshape(muFAprime,(dims[0],dims[1],dims[2]),order = 'F')
muFAprime[mask == 0] = 0
# muFAprime[muFAprime > 1] = 1
# muFAprime[muFAprime < 0] = 0

muFA = np.reshape(muFA,(dims[0],dims[1],dims[2]),order = 'F')
muFA[mask == 0] = 0
# muFA[muFA > 1] = 1
# muFA[muFA < 0] = 0

Re = np.reshape(Re,(dims[0],dims[1],dims[2]),order = 'F')
Re[mask == 0] = 0
# Re[Re < 0] = 0

outname = op.join(dpdkiDir,'Dbar.nii')
newimg = nib.Nifti1Image(Dbar, hdr.affine, hdr.header)
newimg.to_filename(outname)
# nib.save(newimg, outname)

outname = op.join(dpdkiDir,'Wbar.nii')
newimg = nib.Nifti1Image(Wbar, hdr.affine, hdr.header)
newimg.to_filename(outname)
# nib.save(newimg, outname)

outname = op.join(dpdkiDir,'Wbart.nii')
newimg = nib.Nifti1Image(Wbart, hdr.affine, hdr.header)
newimg.to_filename(outname)
# nib.save(newimg, outname)

outname = op.join(dpdkiDir,'WAt.nii')
newimg = nib.Nifti1Image(WAt, hdr.affine, hdr.header)
newimg.to_filename(outname)
# nib.save(newimg, outname)

outname = op.join(dpdkiDir,'FA.nii')
newimg = nib.Nifti1Image(fa, hdr.affine, hdr.header)
nib.save(newimg, outname)

outname = op.join(dpdkiDir,'muFAprime.nii')
newimg = nib.Nifti1Image(muFAprime, hdr.affine, hdr.header)
nib.save(newimg, outname)

outname = op.join(dpdkiDir,'muFA.nii')
newimg = nib.Nifti1Image(muFA, hdr.affine, hdr.header)
nib.save(newimg, outname)

outname = op.join(dpdkiDir,'Re.nii')
newimg = nib.Nifti1Image(Re, hdr.affine, hdr.header)
nib.save(newimg, outname)

### Calculate the associated anisotropies

In [ ]:
# # First the FA

# G = (1/2) * ((AmbL[:,3] - AmbL[:,4])**2 + (AmbL[:,5] - AmbL[:,6])**2 + (AmbL[:,7] - AmbL[:,8])**2) # Eq. 42
# numer = ((AmbL[:,0] - AmbL[:,1])**2 + (AmbL[:,0] - AmbL[:,2])**2 + (AmbL[:,1] - AmbL[:,2])**2 + 3*G)
# denom = ((psi0 - AmbL[:,0])**2 + (psi0 - AmbL[:,1])**2 + (psi0 - AmbL[:,2])**2 + G)

# fa = np.sqrt(numer/(2*denom)) # Eq. 41

# # Type-II microFA
# muFAprime = np.sqrt((30*dWbar)/(9 + 20*dWbar)) #Eq. 39

# # Type-I microFA
# muFA = np.sqrt(3 * (3*fa**2 + 3*muFAprime**2 - 4 * fa**2 * muFAprime**2)/(9 - 4 * fa**2 * muFAprime**2)) #Eq. 40

# fa = np.reshape(fa,(dims[0],dims[1],dims[2]),order = 'F')
# fa[mask == 0] = 0

# muFAprime = np.reshape(muFAprime,(dims[0],dims[1],dims[2]),order = 'F')
# muFAprime[mask == 0] = 0
# muFAprime[muFAprime > 1] = 1
# muFAprime[muFAprime < 0] = 0

# muFA = np.reshape(muFA,(dims[0],dims[1],dims[2]),order = 'F')
# muFA[mask == 0] = 0
# muFA[muFA > 1] = 1
# muFA[muFA < 0] = 0

# outname = op.join(dpdkiDir,'FA.nii')
# newimg = nib.Nifti1Image(fa, hdr.affine, hdr.header)
# nib.save(newimg, outname)

# outname = op.join(dpdkiDir,'muFAprime.nii')
# newimg = nib.Nifti1Image(muFAprime, hdr.affine, hdr.header)
# nib.save(newimg, outname)

# outname = op.join(dpdkiDir,'muFA.nii')
# newimg = nib.Nifti1Image(muFA, hdr.affine, hdr.header)
# nib.save(newimg, outname)


# Calculating the Exchange Rate
### $\Delta$ = 20 ms (diffusion time)
### $\tau$ = 28.1 ms (real mixing time)

In [11]:
# Delta = 24.7 * 10**-3
# tau = 28.1 * 10**-3
# Re = (3/(2*Delta + 3*tau))*np.log(Wbar/WAt)
# Re[mask == 0] = 0
# Re[Re < 0] = 0

# outname = op.join(dpdkiDir,'Re.nii')
# newimg = nib.Nifti1Image(Re, hdr.affine, hdr.header)
# nib.save(newimg, outname)

In [ ]:
print(subj)